In [1]:
from __future__ import print_function
from sklearn.externals import joblib

import matplotlib.pyplot as plt
import numpy as np

from skimage import io
from skimage.segmentation import slic
from skimage.segmentation import mark_boundaries
from skimage.util import img_as_float
from scipy import misc

from PIL import Image
from skimage.measure import regionprops

import sys
import os

# uncomment line below if you use TensorFlow with GPU support, ie CUDA libraries etc
os.environ["CUDA_VISIBLE_DEVICES"]="0" # to allocate only one TensorFLow device

import tensorflow as tf
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions

/home/dvrnd/code/amber_ml/keras_env/local/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [12]:
from lime import lime_image
from lime.wrappers.scikit_image import SegmentationAlgorithm
explainer = lime_image.LimeImageExplainer(verbose = False)
#segmenter = SegmentationAlgorithm('quickshift', kernel_size=1, max_dist=200, ratio=0.2)
segmenter = SegmentationAlgorithm('slic', n_segments=100, compactness=1, sigma=1)


In [2]:
classifier = joblib.load('./versions/xgb_model.pkl')
resnet = ResNet50(weights='imagenet', include_top=False)


In [40]:

def prepare_img(img, target_size=(224, 224)):
    img = img.resize(target_size, Image.NEAREST)
    return img


def get_features(img, resnet, batch_size = 64):
    img_array = np.zeros((1, 224, 224, 3))
    img = preprocess_input(img)
    print( img.shape )
    img_array[0] = img     
    X = resnet.predict(img_array, batch_size=batch_size, verbose=0)
    X = X.reshape((1, 2048))
    return X            

def my_pipeline(img, model=resnet):
    features = get_features(img, model, batch_size = 64)
    return classifier.predict_proba(features)
    
    
    

In [41]:
img_name = "./datasets/positive/51.53218050000052_26.483170000000197_09_Mar_2015_GMT_super_20.png"
img_name = "./datasets/negative/51.51399480000001_27.41170000000028_05_Oct_2015_GMT_super_10.png"
img = prepare_img(Image.open(img_name))
img = img_as_float(img)
#print(img) # for LIME I must convert it to -1, 1
#features = get_features(img, resnet, batch_size = 64)
#Y_pred = classifier.predict(features)
#labels = Y_pred.tolist()
#classifier.predict_proba(features)
my_pipeline(img, resnet)

(224, 224, 3)


array([[0.9984471 , 0.00155287]], dtype=float32)

In [42]:
%%time
explanation = explainer.explain_instance(img, 
                                         classifier_fn = my_pipeline, 
                                         top_labels=2, hide_color=0, num_samples=10000, segmentation_fn=segmenter)


(10, 224, 224, 3)


ValueError: could not broadcast input array from shape (10,224,224,3) into shape (224,224,3)